In [1]:
# Load AIEEE Data

In [2]:
!nvidia-smi

Tue Mar 22 11:26:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0    33W / 250W |  12249MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = ERData_MajorityLabel()

Load Models

In [6]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [ ]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [ ]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [ ]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [ ]:
from transformers import pipeline

In [ ]:
from sklearn.metrics import classification_report

# print(classification_report(true, labels, digits = 4))

In [15]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [ ]:
print_results(cbse_model, tokenizer, er_df)

In [ ]:
print_results(aieee_model, tokenizer, er_df)